In [30]:
# cost_overrun_analysis.py
import pandas as pd
import numpy as np

In [31]:
file_path = "ongoing_projects2_clean.csv"  # change if needed
df = pd.read_csv(file_path)

In [32]:
df.drop(columns=['Sl.No'], inplace=True)

In [33]:
# Normalize column names (adjust if your file uses different names)
df = df.rename(columns=lambda c: c.strip())
# Try to detect the cumulative column name if it contains "Cumulative"
for c in df.columns:
    if "Cumulative" in c:
        df = df.rename(columns={c: "Cumulative_Expenditure_RsCr"})
df = df.rename(columns={
    "Original Cost": "Original_Cost_RsCr",
    "Latest Revised Cost": "Latest_Revised_Cost_RsCr",
    "Project Count": "Project_Count",
    "STATE NAME": "State",
    "Allocated To": "Allocated_To",
    "Sector": "Sector"
})

In [34]:
# Ensure numeric types
num_cols = ["Original_Cost_RsCr", "Latest_Revised_Cost_RsCr", "Cumulative_Expenditure_RsCr", "Project_Count"]
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [35]:
# Compute escalation % safely
df["Cost_Escalation_Pct"] = np.where(
    (df["Original_Cost_RsCr"].notna()) & (df["Original_Cost_RsCr"] != 0),
    (df["Latest_Revised_Cost_RsCr"] - df["Original_Cost_RsCr"]) / df["Original_Cost_RsCr"] * 100,
    np.nan
)

In [36]:
# Top 10 by escalation %
top10 = df.sort_values("Cost_Escalation_Pct", ascending=False).head(10)[
    ["State","Allocated_To","Sector","Project_Count","Original_Cost_RsCr","Latest_Revised_Cost_RsCr","Cost_Escalation_Pct"]
]
print("Top 10 by Cost Escalation %:\n", top10.to_string(index=False))

Top 10 by Cost Escalation %:
             State                                           Allocated_To                          Sector  Project_Count  Original_Cost_RsCr  Latest_Revised_Cost_RsCr  Cost_Escalation_Pct
   Andhra Pradesh Department of Water Resources, River\nDevelopment & GR                 Water Resources              3            10901.50                  57269.50           425.335963
            Assam                                      Ministry of Power          Electricity Generation              1             6285.33                  26075.50           314.862863
           Odisha Department of Water Resources, River\nDevelopment & GR                 Water Resources              2             1556.46                   5291.33           239.959267
Arunachal Pradesh                                   Ministry of Railways                        Railways              1              435.00                   1250.00           187.356322
        Karnataka                  

In [37]:
# Average escalation by sector and by state
sector_avg = df.groupby("Sector", dropna=False)["Cost_Escalation_Pct"].mean().sort_values(ascending=False).reset_index()
state_avg = df.groupby("State", dropna=False)["Cost_Escalation_Pct"].mean().sort_values(ascending=False).reset_index()

In [38]:
print("\nTop sectors by average escalation:\n", sector_avg.head(15).to_string(index=False))
print("\nTop states by average escalation:\n", state_avg.head(15).to_string(index=False))


Top sectors by average escalation:
                             Sector  Cost_Escalation_Pct
   Tourism, Hospitality & Wellness           116.064572
                   Water Resources            92.717800
            Electricity Generation            35.031254
                    Energy Storage            30.261945
                          Railways            30.097782
       Transmission & Distribution            14.383409
                             Steel            11.725541
                        Healthcare             8.218628
                   Metals & Mining             7.143147
                         Oil & Gas             7.040537
            Urban Public Transport             6.550090
Aviation & Aviation Infrastructure             3.114316
                         Education             2.907246
                              Coal             0.609058
                     Waste & Water             0.000000

Top states by average escalation:
             State  Cost_Escalat

In [39]:
# Save outputs
top10.to_csv("cost_overrun_top10.csv", index=False)
sector_avg.to_csv("sector_avg_escalation.csv", index=False)
state_avg.to_csv("state_avg_escalation.csv", index=False)
print("\nSaved: cost_overrun_top10.csv, sector_avg_escalation.csv, state_avg_escalation.csv")


Saved: cost_overrun_top10.csv, sector_avg_escalation.csv, state_avg_escalation.csv


In [40]:
df.sample(10)

,State,Allocated_To,Sector,Project_Count,Cumulative_Expenditure_RsCr,Original_Cost_RsCr,Latest_Revised_Cost_RsCr,Cost_Escalation_Pct
115,Karnataka,Department for Promotion of Industry &\nIntern...,Real Estate,1,88.25,1701.81,1701.81,0.000000
170,Nagaland,Department of Higher Education,Education,1,260.10,275.63,275.63,0.000000
149,Maharashtra,Ministry of Coal,Coal,25,3084.72,12455.88,12455.90,0.000161
72,Gujarat,Ministry of Coal,Electricity Generation,1,303.50,383.84,383.84,0.000000
125,Karnataka,Ministry of Steel,Steel,1,229.77,1043.63,1043.63,0.000000
240,Uttar Pradesh,"Ministry of Ports, Shipping and Waterways",Inland Waterways,1,3510.72,5369.18,5061.15,-5.737003
190,Punjab,Department of Higher Education,Education,2,290.39,559.30,559.30,0.000000
129,Kerala,Ministry of Petroleum & Natural Gas,Oil & Gas,3,4912.37,7996.87,10990.90,37.440023
46,Chhattisgarh,"Department of Water Resources, River\nDevelopm...",Water Resources,2,616.93,842.26,1109.33,31.708736
162,Manipur,Ministry of Railways,Railways,1,15598.11,14323.00,21885.90,52.802486
